### Part 2: Contract Data Binning

Import libraries

In [1]:
import spacy
from spacy.matcher import Matcher
from spacy.pipeline import EntityRuler
from spacy.tokens import Token, Span, Doc
from spacy import displacy
from openpyxl import load_workbook
import numpy as np
import pandas as pd
from dateutil import parser
import wavefunctions as wf
import re
import random
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

Load spacy and pretrained model from pickle

In [2]:
nlp = spacy.load('en_core_web_sm')

nlp.from_disk("")


Load Full Data Set into a pandas dataframe and create a pandas series for each text extract field

In [3]:
full_df = pd.read_excel('data/test/Employee analysis - Full Dataset.xlsx',sheet_name="Full Dataset") # Full Set
train_df = pd.read_excel('data\Employee Train.xlsx',sheet_name="Training Dataset") # Train Set

In [4]:
contract_df = full_df[~(full_df.index.isin(train_df.index))]

In [5]:
times_extract = contract_df['Times [Extract]']
public_extract = contract_df["Work on Public Holidays [Extract]"]
holidays_extract = contract_df["Holiday Entitlement [Extract]"]

Create custom extensions in Spacy to store structured information before adding it to 

In [6]:
# Time extract extensions
Doc.set_extension("specified_days",getter=wf.get_days_per_week_model, force=True)
Doc.set_extension("specified_hours",default=0, force=True)
Doc.set_extension("lunch_hours",default=0,force=True)
Span.set_extension("saved_hours",default=0,force=True)
Span.set_extension("saved_days",default=0,force=True)

#work on public holidays etxensions
Doc.set_extension("work_none",default=0, force=True)
Doc.set_extension("work_normal",default=0, force=True)
Doc.set_extension("work_required",default=0, force=True)
Doc.set_extension("work_easter",default=0, force=True)

#Holiday extensions
Doc.set_extension("hol_30x",default=0, force=True)
Doc.set_extension("dec_jan",default=0, force=True)
Doc.set_extension("leave_as_salary",default=0, force=True)
Doc.set_extension("pro_rata",default=0, force=True)


In [7]:
docs = nlp.pipe(times_extract)

Define preds Dataframe and populate it with predictions from Spacy entities

In [8]:
preds = pd.DataFrame(columns = ["text",
            'Days per week specified', 'Days per week assumed',
            'Days per week blended', 'Hours per week specified',
            'Hours per week assumed', 'Hours per week blended'])
  
for idx,doc in enumerate(docs):
    
    wf.extract_specified_times(doc) # get specified hours from entities and save hours for time ranges for calculation
    wf.get_days_per_week_model(doc) # get specified days per week from entities and save hours for days and day ranges for calculation
    wf.get_hours_from_doc_ents(doc) # complete calculations to derive specified hours from saved hours and days.
    
    spec_days = doc._.specified_days
    spec_hours = doc._.specified_hours
    
    preds.loc[idx] = [doc.text,spec_days,
                      wf.get_assumed(spec_days),
                      wf.get_blended(spec_days),
                      spec_hours,
                      wf.get_assumed(spec_hours,"hours"),
                      wf.get_blended(spec_hours,"hours")]

Checking how well the extraction is performing shows that we are getting classifying 99% of the training data correctly. 

In [9]:
wf.check_results(preds,contract_df)

'98.6% accuracy'

Check the misclassified data, only 1 data row incorrect where information has been reversed

In [10]:
check_preds = preds.drop("text",axis=1)
check_df = contract_df.fillna(0).reset_index(drop=True)[check_preds.columns]

check_preds[~(check_preds == check_df).all(axis=1)]

,Days per week specified,Days per week assumed,Days per week blended,Hours per week specified,Hours per week assumed,Hours per week blended
5,0,5,5,0,37.5,37.5


In [11]:
preds.loc[5,'text']

'Your normal working hours are 20 each week. Your department is generally required to be staffed 08:00 - 18:00hrs, and specific working hours within this will be instructed by your line manager with a one hour lunch break.'

---------------------------------------------

### Extracting Data from the work on public holidays [Extract]

Use Spacy and the Matcher class to allow extra flexibility in matching text patterns

In [12]:
public_matcher = Matcher(nlp.vocab)

In [13]:
def on_normal_match(matcher, doc, id, matches):
      doc._.work_normal = 1
        
def on_required_match(matcher, doc, id, matches):
      doc._.work_required = 1
        
def on_easter_match(matcher, doc, id, matches):
      doc._.work_easter = 1

public_matcher.add("normal_days", on_normal_match, [{"LOWER": "normal"},{"LOWER": "working"}, {"LOWER": "days"}])
public_matcher.add("required_to_work", on_required_match, [{"LOWER": "required"}, {"lower": "to"},{"lower": "work"}])
public_matcher.add("easter_work", on_easter_match, [{"LOWER": "easter"}, {"lower": "revision"}])

preds_public = pd.DataFrame(columns = ["text",
            'Silent re Public Holiday Arrangements', 'Public Holidays during term are normal working days',
            'Public Holidays during Easter Revision course are normal working days', 
            'Working on Public Holidays may be required'])


In [14]:
public_docs = nlp.pipe(public_extract)

In [15]:
for idx,doc in enumerate(public_docs):
    if doc.text == "None":
        doc._.work_none = 1
        
    matches = public_matcher(doc)
    
    preds_public.loc[idx] = [doc.text, doc._.work_none, doc._.work_normal,
                       doc._.work_easter, doc._.work_required]

Simple matching of text phrases gives 100% accuracy, it may be worth reviewing and making this more robust to text/spelling errors

In [16]:
wf.check_results(preds_public,contract_df)

'100.0% accuracy'

-------------

### Extracting Data from the Holiday Entitelment [Extract]

Set up a predictions dataframe for Holiday Entitlement

In [17]:
preds_holiday = pd.DataFrame(columns = [
            'Number of days excluding public holidays expressed if part-time',
            'Number of days including public holidays expressed if part-time',
            'Holiday pro-rata to full-time 25 days (silent as to public holidays)(25)',
            'Holiday pro-rata by hrs to full-time 5.6 weeks including public holidays (or 20+8) (28)',
            'Holiday 6.12 weeks including public holidays (30.x)',
            'Holiday pro-rata to full-time 33 days including 8 public holidays (or 25+8) (33)',
            'Holiday pro-rata to full-time 35 days plus 8 public holidays (43)',
            'Holiday pro-rata to full-time 35 days plus 9 public holidays (44)',
            'Holiday Entitlement paid as part of annual salary',
            'Holiday Entitlement paid Dec and Jun',
            'days_holiday',
            'pro_rata',
            'days_entitled',
            'days_phol'])


Create rules in the matcher to match rule based categorizing several items to be extracted

In [18]:
hol_matcher = Matcher(nlp.vocab)

In [19]:
def on_dec_match(matcher, doc, id, matches):
      doc._.dec_jan = "yes"
        
def on_leave_match(matcher, doc, id, matches):
      doc._.leave_as_salary = "yes"
        
def on_hol30x_match(matcher, doc, id, matches):
      doc._.hol_30x = 1
        
def on_pro_match(matcher, doc, id, matches):
      doc._.pro_rata = 1

hol_matcher.add("dec_jan", on_dec_match, [{"LOWER": "twice"},{"LOWER": "yearly"}, {"LOWER": "in"},{"LOWER": "december"},{"LOWER": "and"}, {"LOWER": "june"}])
hol_matcher.add("leave_as_salary", on_leave_match, [{"LOWER": "leave"}, {"lower": "will"},{"lower": "be"},{"lower": "paid"},{"lower": "as"},{"lower": "part"}])
hol_matcher.add("hol_30x", on_hol30x_match, [{"LOWER": "6.12"}, {"lower": "weeks"}])
hol_matcher.add("pro_rata", on_pro_match, [{"LOWER": "pro"},{"LOWER": "-", "OP": "?"}, {"LOWER": "rata"}])


Get Spacy Doc objects from holiday extract text

In [20]:
hol_docs = nlp.pipe(holidays_extract)

In [21]:
def preprocess(df):
    y = df["Holiday pro-rata to full-time 35 days plus 8 public holidays (43)"].fillna(0) * 6
    is_hol_pro_33 = df["Holiday pro-rata to full-time 33 days including 8 public holidays (or 25+8) (33)"]==1
    is_hol_30x = df["Holiday 6.12 weeks including public holidays (30.x)"]==1
    is_hol_pro_28 = df["Holiday pro-rata by hrs to full-time 5.6 weeks including public holidays (or 20+8) (28)"]==1
    is_hol_pro_44 = df["Holiday pro-rata to full-time 35 days plus 9 public holidays (44)"]==1
    
    y[is_hol_pro_33] = 5 # (33)
    y[is_hol_pro_28] = 3 # (28)
    y[is_hol_pro_44] = 6 # combined with (43) as very similar except for days
    y[is_hol_30x] = 4 # (30.x)
    y[y==0] = 4 # set remaining unclassified example to (30x) as it was not categorized in the xls file  
    y_prep = y[~(y==4)] # remove all (30.x) examples from y that can be categorized from text lookup.
    X_prep = df["Holiday Entitlement [Extract]"][~(y==4)] # remove all (30.x) examples from holidays extract
    
    return X_prep, y_prep

Combine similar and rare categories and correct an unclassified training example

In [22]:
X_prep, y_prep = preprocess(contract_df)

Check that counts of values match those in excel

In [23]:
y_prep.value_counts()

3.0    26
5.0     5
6.0     3
Name: Holiday pro-rata to full-time 35 days plus 8 public holidays (43), dtype: int64

Use pretrained model to predict holiday extract classifications - see end of shee for model training deatils, this was trained with the trinaing data set

In [42]:
DTC_predictions = model.predict(holidays_extract)

In [43]:
one_hot_dtc = pd.get_dummies(DTC_predictions)

In [44]:
for idx,doc in enumerate(hol_docs):
           
    matches = hol_matcher(doc)
    
    preds_holiday.loc[idx] = [0,0,0,0,
                              doc._.hol_30x,
                              0,0,0,
                              doc._.leave_as_salary, 
                              doc._.dec_jan,
                              wf.get_days_holiday(doc),
                              doc._.pro_rata,
                              wf.get_days_entitled(doc),
                              wf.get_days_phol(doc)]

the holiday category (30x) days was not included in the decision tree classifier as there were too few examples and it hurt accuracy, also it seems that it could be easily predicted by a rule so we need to zero predictions from the DTC for examples that can be predicted as 30x by a simple Matcher rule.

In [45]:

hol30x = "Holiday 6.12 weeks including public holidays (30.x)" #4
is_hol30x = (preds_holiday.loc[:,hol30x] == 1)

#clear predictions for examples
one_hot_dtc[is_hol30x] = 0

days_exc = 'Number of days excluding public holidays expressed if part-time' #0
days_inc = 'Number of days including public holidays expressed if part-time' #1
hol25 = 'Holiday pro-rata to full-time 25 days (silent as to public holidays)(25)' #2
hol28 = 'Holiday pro-rata by hrs to full-time 5.6 weeks including public holidays (or 20+8) (28)' #3rd 

hol33 = 'Holiday pro-rata to full-time 33 days including 8 public holidays (or 25+8) (33)' #5
hol43 = 'Holiday pro-rata to full-time 35 days plus 8 public holidays (43)' #6
hol44 = "Holiday pro-rata to full-time 35 days plus 9 public holidays (44)" #7

dec_jan = 'Holiday Entitlement paid Dec and Jun' #8
leave_as_sal = 'Holiday Entitlement paid as part of annual salary' #9

preds_holiday.loc[:,hol28] = one_hot_dtc.loc[:,3]
preds_holiday.loc[:,hol33] = one_hot_dtc.loc[:,5]
preds_holiday.loc[:,hol43] = one_hot_dtc.loc[:,6]
                 

In [47]:
wf.check_results(preds_holiday,contract_df)

'96.9% accuracy'

Calculate pro rata hours whether including bank holidays or excluding bank holidays based on review of data

In [48]:
has_days_inc = ((preds_holiday[hol43]==1) & (preds_holiday["days_holiday"]<43) & 
               (preds_holiday["pro_rata"] ==1) & 
               (preds_holiday["days_entitled"]==43)) | ((preds_holiday[hol33]==1) & 
                (preds_holiday["days_holiday"]<33) &  (preds_holiday["pro_rata"] ==1) & 
                                                       (preds_holiday["days_entitled"]==33)) 

has_days_exc = ((preds_holiday[hol43]==1) & (preds_holiday["days_holiday"]<43) & 
               (preds_holiday["pro_rata"] ==1) & 
               (preds_holiday["days_entitled"]<43)) | ((preds_holiday[hol33]==1) & 
                (preds_holiday["days_holiday"]<33) &  (preds_holiday["pro_rata"] ==1) & 
                                                       (preds_holiday["days_entitled"]<33)) 



In [49]:
preds_holiday.loc[has_days_exc,days_exc] = preds_holiday.loc[has_days_exc,"days_holiday"]
preds_holiday.loc[has_days_inc,days_inc] = preds_holiday.loc[has_days_inc,"days_holiday"]


Seperate hol44 class and hol43 class previously joined together for processing by the DTC, by using nr bank/public holidays included

In [50]:
has_44_days = (preds_holiday[hol43]==1) & (preds_holiday["days_phol"] > 8)

preds_holiday.loc[has_44_days,[hol43,hol44]] = [0,1]

In [52]:
wf.check_results(preds_holiday,contract_df)

'97.7% accuracy'

---------

Combine all 3 prediction dataframes and put them into a copy of the provided dataframe then export to excel

In [62]:
preds_export = pd.concat([preds,preds_public, preds_holiday],axis=1)
preds_export.drop(["text","days_holiday","pro_rata","days_entitled","days_phol"],axis=1,inplace=True)
final_export = contract_df.copy()
final_export[preds_export.columns] = preds_export

In [63]:
with pd.ExcelWriter('Exports\Employee analysis - Full Dataset.xlsx', mode='a') as writer:  
    final_export.to_excel(writer, sheet_name='test predictions',startrow=1,header=False,index=False)

------

In [58]:
check_preds = preds_holiday.drop(['days_holiday', 'days_entitled', 'days_phol', 'pro_rata'],axis=1)
check_df = contract_df.fillna(0).reset_index(drop=True)[check_preds.columns]

check_preds[~(check_preds == check_df).all(axis=1)]

,Number of days excluding public holidays expressed if part-time,Number of days including public holidays expressed if part-time,Holiday pro-rata to full-time 25 days (silent as to public holidays)(25),Holiday pro-rata by hrs to full-time 5.6 weeks including public holidays (or 20+8) (28),Holiday 6.12 weeks including public holidays (30.x),Holiday pro-rata to full-time 33 days including 8 public holidays (or 25+8) (33),Holiday pro-rata to full-time 35 days plus 8 public holidays (43),Holiday pro-rata to full-time 35 days plus 9 public holidays (44),Holiday Entitlement paid as part of annual salary,Holiday Entitlement paid Dec and Jun
0,0,0,0.0,1,0.0,0,0,0.0,0.0,0
1,0,0,0.0,0,0.0,1,0,0.0,0.0,0
7,0,0,0.0,0,0.0,1,0,0.0,0.0,0
31,17,0,0.0,0,0.0,0,1,0.0,0.0,0


In [57]:
check_df[~(check_preds == check_df).all(axis=1)]

,Number of days excluding public holidays expressed if part-time,Number of days including public holidays expressed if part-time,Holiday pro-rata to full-time 25 days (silent as to public holidays)(25),Holiday pro-rata by hrs to full-time 5.6 weeks including public holidays (or 20+8) (28),Holiday 6.12 weeks including public holidays (30.x),Holiday pro-rata to full-time 33 days including 8 public holidays (or 25+8) (33),Holiday pro-rata to full-time 35 days plus 8 public holidays (43),Holiday pro-rata to full-time 35 days plus 9 public holidays (44),Holiday Entitlement paid as part of annual salary,Holiday Entitlement paid Dec and Jun
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
31,17.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0


### Model Training cells moved to end of the notebook for ease of use 
#### use Train_df to train only on training data

Try a bag of words model with TfidfVecotrizer and a Decision Tree Classifier to predict the type of holiday clause for each document

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

consider removing common words via a stop words list to bring down the size of sparse matrices after tfidf processing, check if this improves accuracy as opposed to using max_df for a similar outcome.

In [26]:
stop_words0 = ["you","are","to","at","where","we","will","be","your"]
stop_words1 = ["depending","actually","entitlement","equivalent","calculated","calculating","involve","depending","you","are","entitled","to","taken","at","where","we","require","work","will","be","paid","your"]

In [27]:
tf_vect = TfidfVectorizer(ngram_range = (1,3),stop_words=stop_words1,use_idf=False)

Add categories to the target variable for each of the different common holiday clauses

Pre Process the training set through the TfIdf vectorizer

In [29]:
X_prep,y_prep = preprocess(train_df)

In [30]:
holiday_vectors = tf_vect.fit_transform(X_prep)

Do a train test split to gauge initial accuracy of a tf-idf and DTC based model

In [31]:
X_train, X_test, y_train, y_test = train_test_split(holiday_vectors,y_prep,test_size=0.25)

In [32]:
dtc = DecisionTreeClassifier()

In [33]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [34]:
hol_preds = dtc.predict(X_test)

In [35]:
print(classification_report(y_test,hol_preds))

              precision    recall  f1-score   support

         3.0       0.90      1.00      0.95         9
         5.0       1.00      0.67      0.80         3
         6.0       1.00      1.00      1.00         7

    accuracy                           0.95        19
   macro avg       0.97      0.89      0.92        19
weighted avg       0.95      0.95      0.94        19



Check this accuracy works well across all the data by using cross validation include the tfidf vectorizer params in a gridsearch

Use a pipeline to allow tfidf settings to be tuned at the same time as the DTC

In [36]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('dtc', DecisionTreeClassifier())
])

In [37]:
param_grid = { 
    
    "dtc__max_depth" : [5,7,None],
    "dtc__max_features" : [None,0.8],
    "tfidf__ngram_range" : [(1,3),(1,4)],
    "tfidf__stop_words" : [None,stop_words1],
    # "tfidf__use_idf" : [True, False],
     "tfidf__max_df" : [1.0,0.9],
    # "tfidf__min_df" : [0.0,0.25]
}

gridCV = GridSearchCV(pipeline,param_grid)

In [38]:
gridCV.fit(X_prep,y_prep) # - DO NOT RETRAIN MODEL ON TEST DATA!

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                      

Check the best score from the Grid Search

In [39]:
gridCV.best_score_

0.96

Save the best estimator

In [40]:
model = gridCV.best_estimator_

In [41]:
model.fit(X_prep,y_prep)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['depending', 'actually',
                                             'entitlement', 'equivalent',
                                             'calcul...
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('dtc',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_de

Check detailed Gridsearch results, a large variety of parameters give the same top score of 96 using a decision tree. Manual examination of the cases that fail to be classified properly in split0 could be useful

In [ ]:
pd.DataFrame(gridCV.cv_results_).sort_values(by="mean_test_score",ascending=False)[1:10]

-----------------------

Possible future work , if more samples maybe Spacy text cat or a Deep learning approach with Keras 